# theta vs. hamming distance from source.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
dataset = "full-state-predictions"
filepath = "%s/results/statistics-output.pickle"

# Read file
with open(filepath, "rb") as f:
    dfluxes = pickle.load(f)

In [ ]:
fig, ax = plt.subplots()

means = []
for key, val in dfluxes.items(): 
    y = np.array(val)
    ymean = np.nanmean(y)
    x = np.ones(len(y)) * (key+1)
    xjitter = x + np.random.randn(len(x))* .1
    ax.scatter(xjitter,y, marker="o", color="gray", alpha=1)
    means.append(ymean)

x = range(1, 8)
y = means
ax.plot(x,y)
ax.set_xlabel("hamming distance")
ax.set_ylabel("$\\theta$")
ax.axis([0,8,-0.05,1.05])
#fig.savefig("/Users/Zsailer/Dropbox/epistasis-paper/02_he-comes-from-ensembles/figures/open-walk-divergence.pdf", format="pdf")